In [1]:
!pip install ultralytics

In [66]:
import csv
import os
import json
from ultralytics import YOLO

In [8]:
detectionmodel = YOLO('yolov8n.pt')

In [67]:
def detections(json_file_path):
    """
    Append detected objects and their labels from a JSON file to a CSV catalog.

    Parameters:
    - json_file_path (str): Path to the JSON file containing image paths and labels.
    - detectionmodel: The object representing the YOLO detection model.

    Returns:
    - List of dictionaries containing the detected objects with their labels and confidence.
    """
    # Read JSON file
    with open(json_file_path, 'r') as json_file:
        data = json.load(json_file)

    # Define a list to store the detected objects with their labels and confidence
    detected_objects_list = []

    # Iterate through each entry in the JSON file
    for entry in data:
        # Extract image path and label from JSON data
        image_path = entry['img']
        ground_truth_label = entry['label']

        # Predict with the model
        detection_results = detectionmodel(image_path)
        boxes = detection_results[0].boxes
        class_names = detection_results[0].names

        for box in boxes:
            # Extracting class index, confidence score, and ground truth label
            class_index = box.cls.cpu().numpy()
            confidence = box.conf.cpu().numpy()
            ground_truth_label = [class_names[i] for i in class_index]

            detected_objects_list.append({
                'Image': image_path,
                'Object': ground_truth_label,
                'Confidence': confidence,
                'Ground Truth Label': ground_truth_label
            })

    return detected_objects_list

In [69]:
# Example usage
json_file_path = '/content/json.json'
detectionmodel = YOLO('yolov8n.pt')
detected_objects = detections(json_file_path)
print(detected_objects)


image 1/1 /content/01235.png: 448x640 1 person, 197.0ms
Speed: 4.1ms preprocess, 197.0ms inference, 1.9ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /content/01236.png: 640x576 6 persons, 1 horse, 6 sheeps, 237.2ms
Speed: 4.8ms preprocess, 237.2ms inference, 4.3ms postprocess per image at shape (1, 3, 640, 576)
[{'Image': '/content/01235.png', 'Object': ['person'], 'Confidence': array([    0.96653], dtype=float32), 'Ground Truth Label': ['person']}, {'Image': '/content/01236.png', 'Object': ['person'], 'Confidence': array([    0.75815], dtype=float32), 'Ground Truth Label': ['person']}, {'Image': '/content/01236.png', 'Object': ['person'], 'Confidence': array([    0.70568], dtype=float32), 'Ground Truth Label': ['person']}, {'Image': '/content/01236.png', 'Object': ['horse'], 'Confidence': array([    0.58712], dtype=float32), 'Ground Truth Label': ['horse']}, {'Image': '/content/01236.png', 'Object': ['person'], 'Confidence': array([    0.50091], dtype=float32), 'Groun

In [71]:
# Assuming your JSON file is named 'data.json'
json_file_path = 'json.json'

# Open the JSON file and read its contents
with open(json_file_path, 'r') as file:
    json_data = json.load(file)

# Now json_data contains the JSON data from the file

csv_data = detected_objects

# Create a dictionary to map image paths to labels
label_map = {data['img']: data['label'] for data in json_data}

# Update Ground Truth Label in csv_data
for data in csv_data:
    img_path = data['Image']
    if img_path in label_map:
        data['Ground Truth Label'] = label_map[img_path]

# Specify the file name
file_name = "./data.csv"

# Specify the fieldnames
fieldnames = ['Image', 'Object', 'Confidence', 'Ground Truth Label']

# Remove the existing file if it exists
if os.path.exists(file_name):
    os.remove(file_name)

# Writing data to CSV file
with open(file_name, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    # Write data rows
    writer.writerows(csv_data)

print("Data written to CSV file successfully.")


Data written to CSV file successfully.


In [72]:
import csv

def load_csv_data(csv_file_path):
    """
    Load data from a CSV file.

    Parameters:
    - csv_file_path (str): Path to the CSV file.

    Returns:
    - List of dictionaries containing the data from the CSV file.
    """
    data = []

    # Read data from CSV file
    with open(csv_file_path, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            data.append(row)

    return data

# Example usage
csv_file_path = '/content/data.csv'
csv_data = load_csv_data(csv_file_path)

# Print the loaded data
for row in csv_data:
    print(row)

{'Image': '/content/01235.png', 'Object': "['person']", 'Confidence': '[    0.96653]', 'Ground Truth Label': 'hateful'}
{'Image': '/content/01236.png', 'Object': "['person']", 'Confidence': '[    0.75815]', 'Ground Truth Label': 'non-hateful'}
{'Image': '/content/01236.png', 'Object': "['person']", 'Confidence': '[    0.70568]', 'Ground Truth Label': 'non-hateful'}
{'Image': '/content/01236.png', 'Object': "['horse']", 'Confidence': '[    0.58712]', 'Ground Truth Label': 'non-hateful'}
{'Image': '/content/01236.png', 'Object': "['person']", 'Confidence': '[    0.50091]', 'Ground Truth Label': 'non-hateful'}
{'Image': '/content/01236.png', 'Object': "['sheep']", 'Confidence': '[    0.48118]', 'Ground Truth Label': 'non-hateful'}
{'Image': '/content/01236.png', 'Object': "['person']", 'Confidence': '[    0.46822]', 'Ground Truth Label': 'non-hateful'}
{'Image': '/content/01236.png', 'Object': "['sheep']", 'Confidence': '[    0.46252]', 'Ground Truth Label': 'non-hateful'}
{'Image': '/con